In [ ]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import speech_recognition as sr
from googletrans import Translator
from twilio.rest import Client

In [ ]:
# List available devices and set a valid input device
def list_audio_devices():
    print(sd.query_devices())

In [ ]:
def get_voice_input():
    samplerate = 44100  # Sampling rate
    duration = 5  # Duration in seconds
    print("Listening for symptoms... Speak now!")
    try:
        audio_data = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=1, dtype='int16')
        sd.wait()  # Wait for the recording to finish
        wav.write("temp_audio.wav", samplerate, audio_data)  # Save as WAV file
        recognizer = sr.Recognizer()
        with sr.AudioFile("temp_audio.wav") as source:
            audio = recognizer.record(source)
        text = recognizer.recognize_google(audio)
        print("User said:", text)
        return text
    except Exception as e:
        print("Error capturing audio:", e)
        return None

In [ ]:
def translate_text(text, target_lang='en'):
    translator = Translator()
    translated = translator.translate(text, dest=target_lang)
    return translated.text

In [ ]:
# Twilio configuration
TWILIO_ACCOUNT_SID = "your_account_sid"
TWILIO_AUTH_TOKEN = "your_auth_token"
TWILIO_WHATSAPP_NUMBER = "whatsapp:+14155238886"
YOUR_PHONE_NUMBER = "whatsapp:+your_phone_number"

In [ ]:
def send_whatsapp_message(message):
    client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
    try:
        client.messages.create(
            from_=TWILIO_WHATSAPP_NUMBER,
            body=message,
            to=YOUR_PHONE_NUMBER
        )
        print("Message sent successfully!")
    except Exception as e:
        print("Failed to send message:", e)

In [ ]:
def main():
    list_audio_devices()  # List devices before recording
    text = get_voice_input()
    if text:
        translated_text = translate_text(text)
        print("Translated:", translated_text)
        send_whatsapp_message(translated_text)

In [ ]:
if __name__ == "__main__":
    main()